In [ ]:
import numpy as np
import random
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

def create_clients(Features_list, Targets_list, num_clients=10, initial='clients'):
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
       
    #shard data and place at each client
    size = len(df)//num_clients
    shards = [df[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))
    return {client_names[i] : shards[i] for i in range(len(client_names))}
def batch_data(data_shard, bs=32):
    #seperate shard into two lists
    data = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data)))
    return dataset.shuffle(len(Features_list)).batch(bs)

class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(64, activation= "relu",input_shape=(9,)))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation= "relu"))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation= "relu"))
        return model

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client    local_count=tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

def test_model(X_test, Y_test,  model, comm_round):
    mse = tf.keras.losses.MeanAbsoluteError()
    y_pred = model.predict(X_test)
    loss = mse(y_test,y_pred)
    print('global_loss:',loss.numpy())
    return loss